In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.inspection import permutation_importance
from fairlearn.metrics import MetricFrame
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, demographic_parity_ratio 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing 
from aif360.algorithms.inprocessing import AdversarialDebiasing
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import tensorflow as tf
import warnings


pd.set_option("display.max_columns", None)
warnings.simplefilter(action='ignore', category=FutureWarning)

if tf.__version__.startswith('2'):
    tf.compat.v1.disable_eager_execution


c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\steve\Clio_Muse Data Analysis Project\.conda\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migratin

In [3]:
df = pd.read_csv('diabetic_data.csv')
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [4]:
df = df[ (df['race'] != '?') & (df['gender'] != '?') & (df['gender'] != 'Unknown/Invalid') & (df['race'] != 'Other')] 

In [5]:
a = df['readmitted'].unique()
print(a)

['NO' '>30' '<30']


In [6]:
df['readmitted'] = df['readmitted'].apply(lambda x: 0 if x == 'NO' else 1)

In [7]:
df = df.drop(columns=['encounter_id', 'patient_nbr','payer_code'])

In [9]:
def convert_weight(value):
    if pd.isna(value) or value == '?':
        return np.nan
    elif '[' in value and ')' in value:
        lower, upper = value.strip('[]()').split('-')
        return (float(lower) + float(upper)) / 2
    elif '>' in value:
        a = value.strip('>')
        return int(a)
    else:
        return float(value)

df['weight'] = df['weight'].apply(convert_weight)

In [10]:
df['age'].fillna(df['age'].mode(), inplace=True)

In [11]:
print(df.dtypes)

race                         object
gender                       object
age                          object
weight                      float64
admission_type_id             int64
discharge_disposition_id      int64
admission_source_id           int64
time_in_hospital              int64
medical_specialty            object
num_lab_procedures            int64
num_procedures                int64
num_medications               int64
number_outpatient             int64
number_emergency              int64
number_inpatient              int64
diag_1                       object
diag_2                       object
diag_3                       object
number_diagnoses              int64
max_glu_serum                object
A1Cresult                    object
metformin                    object
repaglinide                  object
nateglinide                  object
chlorpropamide               object
glimepiride                  object
acetohexamide                object
glipizide                   

In [ ]:
categorical = [
    'race',
    'gender',
    'age',
    'medical_specialty',
    'max_glu_serum',
    'A1Cresult',
    'change',
    'diag_1',
    'diag_2',
    'diag_3',
    'metformin',
    'repaglinide',
    'nateglinide',
    'chlorpropamide',
    'glimepiride',
    'acetohexamide',
    'glipizide',
    'glyburide',
    'tolbutamide',
    'pioglitazone',
    'rosiglitazone',
    'acarbose',
    'miglitol',
    'troglitazone',
    'tolazamide',
    'examide',
    'citoglipton',
    'insulin',
    'glyburide-metformin',
    'glipizide-metformin',
    'glimepiride-pioglitazone',
    'metformin-rosiglitazone',
    'metformin-pioglitazone',
    'diabetesMed'
]

numerical = [
    'weight',
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'time_in_hospital',
    'number_outpatient',
    'number_inpatient',
    'number_emergency',
    'num_lab_procedures',
    'number_diagnoses',
    'num_medications',
    'num_procedures'
    #'payer_code',
]

In [16]:
for col in df.columns:
    if col not in numerical:
        if col not in categorical:
            print(col, df[col].dtype)
            if col != 'readmitted':
                categorical.append(col)
            

readmitted int64


In [19]:
print(df[categorical].isna().sum())


race                     0
gender                   0
age                      0
medical_specialty        0
max_glu_serum        92767
A1Cresult            81696
change                   0
dtype: int64


In [20]:
print(df[numerical].isna().sum())

weight                      94958
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
number_outpatient               0
number_inpatient                0
number_emergency                0
num_lab_procedures              0
number_diagnoses                0
num_medications                 0
num_procedures                  0
dtype: int64


In [21]:
df[numerical] = df[numerical].replace('?', np.nan)
df[categorical] = df[categorical].replace('?', np.nan)


df[numerical] = df[numerical].fillna(df[numerical].mean())
df[categorical] = df[categorical].fillna(df[categorical].mode().iloc[0])

In [22]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical),
        ("cat", Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical)
    ]
)

In [23]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter = 1000,random_state=42))
])

In [24]:
X = df.drop('readmitted', axis=1)
y = df['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)






pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))



Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.79      0.69     15721
           1       0.63      0.41      0.50     13676

    accuracy                           0.62     29397
   macro avg       0.62      0.60      0.59     29397
weighted avg       0.62      0.62      0.60     29397

ROC-AUC Score: 0.6521855801605128


In [25]:
def get_rf_pipeline_without_encoding():
    #this function returns a random forest classifier pipeline
    numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

                                    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)
    ])

    pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42))
    ])
    return pipeline_rf

In [26]:
def get_rf_pipeline():
    #this function returns a random forest classifier pipeline
    numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

                                    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)
    ])

    pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42))
    ])
    return pipeline_rf

In [27]:
pipeline_rf = get_rf_pipeline()

X = df.drop('readmitted', axis=1)
y = df['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

pipeline_rf.fit(X_train, y_train)

y_pred = pipeline_rf.predict(X_test)
y_pred_proba = pipeline_rf.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.70      0.67     15721
           1       0.61      0.53      0.57     13676

    accuracy                           0.62     29397
   macro avg       0.62      0.62      0.62     29397
weighted avg       0.62      0.62      0.62     29397

ROC-AUC Score: 0.666365563345288


In [28]:
groups = ['gender', 'race']
results = []
privileged = {}
for group in groups:
    privileged[group] = {}

In [29]:
def evaluate_fairness(y_true, y_pred, sensitive_features, group_name):
    eod = equalized_odds_difference(
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features
    )
    
    dpd = demographic_parity_difference(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    di_ratio = demographic_parity_ratio(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features
    )
    
    print(f'\n group is {group_name}')
    print(f"Demographic Parity Ratio: {di_ratio:.4f}")
    print(f"Equalized Odds Difference: {eod:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    
    
    positive_rates = {}
    for group_value in sensitive_features.unique():
        mask = sensitive_features == group_value
        group_y_pred = y_pred[mask]
        positive_rate = group_y_pred.mean()
        positive_rates[group_value] = positive_rate
        print(f"Subgroup: {group_value}, Positive Prediction Rate: {positive_rate:.4f}")
    
    max_rate = max(positive_rates.values())
    min_rate = min(positive_rates.values())
    
    positive_rates = dict(sorted(positive_rates.items(), key=lambda x: x[1]))
    
    
    values = list(positive_rates.values())
    q1, q3 = np.percentile(values, [25, 75])
    iqr = q3 - q1
    threshold = iqr * 0.5 
        
    
    for group_value, rate in positive_rates.items():
        
        if rate == min_rate or (rate - min_rate <= threshold):
            print(f"--> Privileged Group: {group_value} (Positive Rate: {rate:.4f})")
            
            if 'privileged' in privileged[group_name]:
                privileged[group_name]['privileged'].append(group_value)
            else:
                privileged[group_name]['privileged'] = [group_value]

        elif rate == max_rate:
            
            print(f"--> Unprivileged Group: {group_value} (Positive Rate: {rate:.4f})")
            if 'unprivileged' in privileged[group_name]:
                privileged[group_name]['unprivileged'].append(group_value)
            else:
                privileged[group_name]['unprivileged'] = [group_value]

In [30]:
for group in groups:
    evaluate_fairness(y_test, y_pred, X_test[group], group)


 group is gender
Demographic Parity Ratio: 0.9058
Equalized Odds Difference: 0.0422
Demographic Parity Difference: 0.0400
Subgroup: Female, Positive Prediction Rate: 0.4242
Subgroup: Male, Positive Prediction Rate: 0.3842
--> Privileged Group: Male (Positive Rate: 0.3842)
--> Unprivileged Group: Female (Positive Rate: 0.4242)

 group is race
Demographic Parity Ratio: 0.5936
Equalized Odds Difference: 0.2216
Demographic Parity Difference: 0.1685
Subgroup: AfricanAmerican, Positive Prediction Rate: 0.3853
Subgroup: Caucasian, Positive Prediction Rate: 0.4147
Subgroup: Asian, Positive Prediction Rate: 0.2462
Subgroup: Hispanic, Positive Prediction Rate: 0.3247
--> Privileged Group: Asian (Positive Rate: 0.2462)
--> Unprivileged Group: Caucasian (Positive Rate: 0.4147)


In [ ]:

for group in groups:
    
    aif_dict = {}

    for element in privileged[group]['privileged']:    
        aif_dict[element] = 1
    
    for element in privileged[group]['unprivileged']:
        aif_dict[element] = 0


    privileged_class = [key for key, value in aif_dict.items() if value == 1]
    unprivileged_class = [key for key, value in aif_dict.items() if value == 0]

    print(f'privileged classes for group {group} are {privileged_class}')
    print(f'unprivileged classes for group {group} are {unprivileged_class}')
    
    #map column values to 0,1 s, based on whether or not the entry is privileged
    #df[group] = df[group].apply(lambda x: 1 if x in(privileged_class) else 0)


encoder = LabelEncoder()


for col in categorical:
    
    df[col] = encoder.fit_transform(df[col].astype(str))

    print(f"Encoded {col}: {dict(zip(encoder.classes_, range(len(encoder.classes_))))}")



all_unique_values = set()

for col in categorical:
    all_unique_values.update(df[col].astype(str).unique())


privileged classes for group gender are ['Male']
unprivileged classes for group gender are ['Female']
privileged classes for group race are ['Asian']
unprivileged classes for group race are ['Caucasian']
Encoded race: {'AfricanAmerican': 0, 'Asian': 1, 'Caucasian': 2, 'Hispanic': 3}
Encoded gender: {'Female': 0, 'Male': 1}
Encoded age: {'[0-10)': 0, '[10-20)': 1, '[20-30)': 2, '[30-40)': 3, '[40-50)': 4, '[50-60)': 5, '[60-70)': 6, '[70-80)': 7, '[80-90)': 8, '[90-100)': 9}
Encoded medical_specialty: {'AllergyandImmunology': 0, 'Anesthesiology': 1, 'Anesthesiology-Pediatric': 2, 'Cardiology': 3, 'Cardiology-Pediatric': 4, 'DCPTEAM': 5, 'Dentistry': 6, 'Dermatology': 7, 'Emergency/Trauma': 8, 'Endocrinology': 9, 'Endocrinology-Metabolism': 10, 'Family/GeneralPractice': 11, 'Gastroenterology': 12, 'Gynecology': 13, 'Hematology': 14, 'Hematology/Oncology': 15, 'Hospitalist': 16, 'InfectiousDiseases': 17, 'InternalMedicine': 18, 'Nephrology': 19, 'Neurology': 20, 'Neurophysiology': 21, 'Ob

In [32]:
scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])


df.head(5)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2,0,0,3.106915e-15,2.740623,4.000630,-1.170204,-1.138710,37,-0.105086,-0.78615,-1.853242,-0.292645,-0.214059,-0.506680,250.83,?,?,-3.348557,2,1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,No,0
1,2,0,1,3.106915e-15,-0.707094,-0.516886,0.301310,-0.468947,18,0.810039,-0.78615,0.241579,-0.292645,-0.214059,-0.506680,276,250.01,255,0.809140,2,1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,0,Yes,1
2,0,0,2,3.106915e-15,-0.707094,-0.516886,0.301310,-0.803828,18,-1.630296,2.15245,-0.374545,1.267497,-0.214059,0.277649,648,250,V27,-0.749996,2,1,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,Yes,0
3,2,1,3,3.106915e-15,-0.707094,-0.516886,0.301310,-0.803828,18,0.047435,-0.19843,-0.004871,-0.292645,-0.214059,-0.506680,8,250.43,403,-0.230284,2,1,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,0,Yes,0
4,2,1,4,3.106915e-15,-0.707094,-0.516886,0.301310,-1.138710,18,0.403317,-0.78615,-0.990669,-0.292645,-0.214059,-0.506680,197,157,250,-1.269709,2,1,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,0,Yes,0


In [34]:
df.dtypes

race                          int32
gender                        int32
age                           int32
weight                      float64
admission_type_id           float64
discharge_disposition_id    float64
admission_source_id         float64
time_in_hospital            float64
medical_specialty             int32
num_lab_procedures          float64
num_procedures              float64
num_medications             float64
number_outpatient           float64
number_emergency            float64
number_inpatient            float64
diag_1                       object
diag_2                       object
diag_3                       object
number_diagnoses            float64
max_glu_serum                 int32
A1Cresult                     int32
metformin                    object
repaglinide                  object
nateglinide                  object
chlorpropamide               object
glimepiride                  object
acetohexamide                object
glipizide                   

In [33]:
dataset = StandardDataset(
    df,
    label_name='readmitted',
    favorable_classes=[0],
    protected_attribute_names=['race', 'gender'],
    privileged_classes=[[1,3], [1]]
    # categorical_features=categorical_columns,
    # features_to_drop=features_to_drop,
    #na_values=na_values
)

ValueError: could not convert string to float: 'V57'


ValueError: DataFrame values must be numerical.